In [3]:
import torch
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)
# load pretrained models, using ResNeSt-50 as an example
model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
model.eval()

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /root/.cache/torch/hub/master.zip
Using cache found in /root/.cache/torch/hub/zhanghang1989_ResNeSt_master


ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): SplAtConv2d(
        (conv): Conv2d

In [4]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [5]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))


/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


tensor([ 2.4271e-01, -1.7915e-01, -1.6784e-01,  1.4744e-01, -3.2213e-01,
        -4.1240e-02, -1.9824e-02,  2.8830e-01,  7.2640e-01, -1.4942e-01,
        -2.8938e-01, -5.2042e-02,  2.7963e-01, -1.4311e-01,  2.8453e-01,
        -5.7207e-02, -2.9288e-01, -1.1528e-01, -4.2813e-01, -4.1283e-01,
        -1.2124e-01,  2.0144e-02, -7.0579e-02, -7.0398e-01, -5.8025e-01,
        -4.2704e-02,  1.3230e-01,  3.9979e-03, -2.0859e-01, -4.9548e-01,
        -5.5671e-03,  6.8378e-02,  3.0727e-02, -2.3538e-01, -2.3623e-01,
        -2.1838e-01, -1.4642e-02, -3.6024e-02,  1.0105e-02,  3.2280e-02,
        -8.9886e-03, -1.3900e-01, -1.7709e-01, -1.5366e-01, -4.2541e-01,
        -6.7351e-03,  1.1944e-01, -7.9377e-03,  1.4479e-01, -3.9386e-02,
        -7.9602e-02, -8.1164e-02,  6.4051e-04, -2.0356e-01, -1.7461e-01,
         2.7610e-03, -3.0141e-01, -4.2637e-02, -3.5680e-01, -8.1847e-02,
         7.0873e-02, -7.9611e-02, -8.2599e-03, -3.2718e-01,  2.7305e-01,
        -1.9174e-01,  4.9259e-01, -6.5162e-02,  1.8

In [13]:
import matplotlib 
from matplotlib import pyplot as plt
def save_image(numpy_array, name):
    '''
    numpy array 형태의 이미지를 받아서 png파일로 저장하는 함수
    
    '''
    matplotlib.image.imsave(name, numpy_array)
    

In [14]:
from PIL import Image
im = Image.fromarray(output[0].numpy())
if im.mode != 'RGB':
    im = im.convert('RGB')
im.save("your_file.png")